In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import pickle
from torch_implementation.language_model import LanguageModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, BatchSampler
import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [7]:
TF_ENABLE_ONEDNN_OPTS=0
torch.set_float32_matmul_precision('medium')

In [3]:
tokenizer = "tokenizer_5000"

with open(f"data/tokenized_data/train_{tokenizer}_data.pkl", "rb") as file:
    train_data = pickle.load(file)
# with open(f"data/tokenized_data/test_{tokenizer}_data.pkl", "rb") as file:
#     test_data = pickle.load(file)

train_data = [token for sents in train_data for token in sents]
# test_data = [token for sents in test_data for token in sents]

In [4]:
train_data = torch.tensor(train_data)
# test_data = torch.tensor(test_data)

In [8]:
train_dt = TensorDataset(train_data[:-1], train_data[1:])
seq_sampler = SequentialSampler(train_dt)
# rand_sampler = RandomSampler(train_dt, num_samples=3)
batch_sampler = BatchSampler(seq_sampler, batch_size=128*256, drop_last=True)
train_loader = DataLoader(train_dt, num_workers=10, batch_sampler=batch_sampler, persistent_workers=True)#, batch_size=128*256)

model = LanguageModel(
    vocab_size=5000,
    n_embed=256,
    context_len=256,
    n_blocks=6,
    n_heads=8,
    n_experts=4,
    top_k=2,
    lr=3e-4,
).bfloat16()

In [9]:
trainer = L.Trainer(max_epochs=10, devices=1, accelerator='gpu')
trainer.fit(model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params | Mode 
---------------------------------------------------------------
0 | embed_layer      | Embedding         | 1.3 M  | train
1 | positional_embed | Embedding         | 65.5 K | train
2 | blocks           | TransformerBlocks | 7.9 M  | train
3 | layer_norm       | LayerNorm         | 512    | train
4 | llm_head         | Linear            | 1.3 M  | train
---------------------------------------------------------------
10.5 M    Trainable params
0         Non-trainable params
10.5 M    Total params
42.137    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Exception ignored in: <function _xla_gc_callback at 0x7ff10ce38b80>
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
